<a href="https://colab.research.google.com/github/sitivio/vio/blob/main/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import xgboost as xgb

In [ ]:
data = pd.read_csv('data_scraping.csv', sep=',', encoding='utf-8')
data.head()

,comment,value
0,"Setelah update, justru ada masalah baru. Waktu...",Positif
1,Tolong di perbaiki bug nya. Kesel bngt iklan m...,Negatif
2,Super... Kita dapat menemukan konten sesuai mi...,Positif
3,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,Negatif
4,"Ini mulai ada iklan, menurut aku sih gapapa ka...",Negatif


In [ ]:
#presos case folding
import re
def casefolding(comment):
  comment = comment.lower()
  comment = comment.strip(" ")
  comment = re.sub(r'[?|$|.|!2_:")(-+,]', '', comment)
  return comment
data['comment_casefolding'] = data['comment'].apply(casefolding)
data.head()

,comment,value,comment_casefolding
0,"Setelah update, justru ada masalah baru. Waktu...",Positif,setelah update justru ada masalah baru waktu m...
1,Tolong di perbaiki bug nya. Kesel bngt iklan m...,Negatif,tolong di perbaiki bug nya kesel bngt iklan mu...
2,Super... Kita dapat menemukan konten sesuai mi...,Positif,super kita dapat menemukan konten sesuai minat...
3,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,Negatif,kecewa ahh sekarang iklan tik tok kok ga berhe...
4,"Ini mulai ada iklan, menurut aku sih gapapa ka...",Negatif,ini mulai ada iklan menurut aku sih gapapa kar...


In [ ]:
#proses tokenizing
def token(comment):
  nstr = comment.split(' ')
  dat= []
  a = -1
  for hu in nstr:
    a = a + 1
  if hu == '':
    dat.append(a)
  p = 0
  b = 0
  for q in dat:
    b = q - p
    del nstr[b]
    p = p + 1
  return nstr
data['comment_tokens'] = data['comment_casefolding'].apply(token)
data.head()

,comment,value,comment_casefolding,comment_tokens
0,"Setelah update, justru ada masalah baru. Waktu...",Positif,setelah update justru ada masalah baru waktu m...,"[setelah, update, justru, ada, masalah, baru, ..."
1,Tolong di perbaiki bug nya. Kesel bngt iklan m...,Negatif,tolong di perbaiki bug nya kesel bngt iklan mu...,"[tolong, di, perbaiki, bug, nya, kesel, bngt, ..."
2,Super... Kita dapat menemukan konten sesuai mi...,Positif,super kita dapat menemukan konten sesuai minat...,"[super, kita, dapat, menemukan, konten, sesuai..."
3,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,Negatif,kecewa ahh sekarang iklan tik tok kok ga berhe...,"[kecewa, ahh, sekarang, iklan, tik, tok, kok, ..."
4,"Ini mulai ada iklan, menurut aku sih gapapa ka...",Negatif,ini mulai ada iklan menurut aku sih gapapa kar...,"[ini, mulai, ada, iklan, menurut, aku, sih, ga..."


In [ ]:
!pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=e5fe2688aa826fae87ea1b34b8cc7325b7e51214575336de5f1da41139ea8674
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [ ]:
#proses filtering
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(comment):
  filtering = stopwords.words('indonesian','english')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, comment)
  for x in fit:
    data.append(x)
  return data
data['comment_filtering'] = data['comment_tokens'].apply(stopword_removal)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,comment,value,comment_casefolding,comment_tokens,comment_filtering
0,"Setelah update, justru ada masalah baru. Waktu...",Positif,setelah update justru ada masalah baru waktu m...,"[setelah, update, justru, ada, masalah, baru, ...","[update, menonton, video, iklan, muncul, setal..."
1,Tolong di perbaiki bug nya. Kesel bngt iklan m...,Negatif,tolong di perbaiki bug nya kesel bngt iklan mu...,"[tolong, di, perbaiki, bug, nya, kesel, bngt, ...","[tolong, perbaiki, bug, nya, kesel, bngt, ikla..."
2,Super... Kita dapat menemukan konten sesuai mi...,Positif,super kita dapat menemukan konten sesuai minat...,"[super, kita, dapat, menemukan, konten, sesuai...","[super, menemukan, konten, sesuai, minat, baka..."
3,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,Negatif,kecewa ahh sekarang iklan tik tok kok ga berhe...,"[kecewa, ahh, sekarang, iklan, tik, tok, kok, ...","[kecewa, ahh, iklan, tik, tok, ga, berhenti, k..."
4,"Ini mulai ada iklan, menurut aku sih gapapa ka...",Negatif,ini mulai ada iklan menurut aku sih gapapa kar...,"[ini, mulai, ada, iklan, menurut, aku, sih, ga...","[iklan, sih, gapapa, , detik, doang, skip, tao..."


In [ ]:
import re

def cleancomment(comment):
    comment = re.sub('@[A-Za-z0–9]+', '', comment) #Removing @mentions
    comment = re.sub('#', '', comment) # Removing '#' hash tag
    comment = re.sub('RT[\s]+', '', comment) # Removing RT
    comment = re.sub('https?:\/\/\S+', '', comment) # Removing hyperlink
    comment = re.sub(r'\d+', '', comment)
    return comment

data['comment'] = data['comment'].apply(cleancomment)



def remove_emoji(cleancomment):
   return clean(comment, no_emoji=True)

data['comment_cleans'] = data['comment_filtering'].apply(remove_emoji)

def remove_emoji(comment):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticon
                               u"\U0001F300-\U0001F5FF"  # simbol & emoji lainnya
                               u"\U0001F680-\U0001F6FF"  # transportasi & ikon map
                               u"\U0001F1E0-\U0001F1FF"  # flag negara
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', comment)

data['comment_cleans'] = data['comment_filtering'].apply(remove_emoji)

def remove_numbers(comment):
    # Menggunakan ekspresi reguler untuk mencocokkan angka dan menghapusnya
    return re.sub(r'\d+', '', comment)
# Contoh penggunaan
comment_with_numbers = "Ini adalah contoh kalimat dengan angka 123 dan 4567."
comment_without_numbers = remove_numbers(comment_with_numbers)
print("Teks awal:", comment_with_numbers)
print("Teks tanpa angka:", comment_without_numbers)

data['comment_tokens'] = data['comment_filtering'].apply(remove_numbers)

NameError: name 'clean' is not defined

In [ ]:
!pip install swifter
!pip install PySastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Fungsi stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

data['comment_tokens'] = data['comment'].apply(stemmed_wrapper)

# Stemming pada setiap kata
data['comment_stemmed'] = data['comment_tokens'].apply(stemmed_wrapper)

print(data[['comment', 'comment_tokens', 'comment_stemmed']])


                                                comment  \
0     Setelah update, justru ada masalah baru. Waktu...   
1     Tolong di perbaiki bug nya. Kesel bngt iklan m...   
2     Super... Kita dapat menemukan konten sesuai mi...   
3     Kecewa ahh.. Sekarang iklan tik tok kok ga ber...   
4     Ini mulai ada iklan, menurut aku sih gapapa ka...   
...                                                 ...   
1195  Setelah update maleh goblok,Mendo,pekok benakn...   
1196  Makin kesini makin ngelag. Hpku sampe mati mat...   
1197                    Pengen Akun tik tok ku rame min   
1198  Tiktok babi memang kalian, saya komen hal yang...   
1199                        Jangan lupa folow LAL Stroy   

                                         comment_tokens  \
0     telah update justru ada masalah baru waktu ton...   
1     tolong di baik bug nya kesel bngt iklan muncul...   
2     super kita dapat temu konten sesuai minat dan ...   
3     kecewa ahh sekarang iklan tik tok kok ga henti...

In [ ]:
data.head()

,comment,value
0,"Setelah update, justru ada masalah baru. Waktu...",Positif
1,Tolong di perbaiki bug nya. Kesel bngt iklan m...,Negatif
2,Super... Kita dapat menemukan konten sesuai mi...,Positif
3,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,Negatif
4,"Ini mulai ada iklan, menurut aku sih gapapa ka...",Negatif


In [ ]:
data.to_csv('data_preprocessing.csv')

In [ ]:
import pandas as pd
import numpy as np

data2 = pd.read_csv("data_preprocessing.csv", usecols=["comment_stemmed"])
data2.columns = ["comment_stemmed"]

data2.head()

,comment_stemmed
0,telah update justru ada masalah baru waktu ton...
1,tolong di baik bug nya kesel bngt iklan muncul...
2,super kita dapat temu konten sesuai minat dan ...
3,kecewa ahh sekarang iklan tik tok kok ga henti...
4,ini mulai ada iklan turut aku sih gapapa karen...


In [ ]:
!pip3 install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
